# Truck Data Wrangler

Exploratory analysis of the dataset inside the `data` folder.

## Unifying the csv´s

First of all we will aggregate all csv data inside a single one. All the CSV files has these commons columns:

 - Timestamp (`timestamp timestamp`)
 - Longitudinal acceleration (`accel_x decimal (12, 6)`)
 - Lateral acceleration (`accel_y decimal (12, 6)`)
 - Vertical acceleration (`accel_z decimal (12, 6)`)
 - Roll rotation (`gyro_roll decimal (12, 6)`)
 - Pitch rotation (`gyro_pitch decimal (12, 6)`)
 - Yaw rotation (`gyro_yaw decimal (12, 6)`)
 - Label (`label integer`)

And we will create another one to identify from which file the rows are coming from:

 - Event type (`event_type string`)
 
Altought this is not an scalable approach, I will use pandas just for the sake of this specific dataset analysis.

In [1]:
import pandas as pd

data_files = {
    'aggressive_bump_1550163148318484.csv': 'agressive_bump',
    'aggressive_longitudinal_acceleration_1549653321089461.csv': 'aggressive_longitudinal_acceleration',
    'aggressive_turn_1549625320507325.csv': 'aggressive_turn',
    'normal_longitudinal_acceleration_1549908723215048.csv': 'normal_longitudinal_acceleration',
    'normal_mixed_1549901031015048.csv': 'normal_mixed',
    'normal_mixed_1550054269957615.csv': 'normal_mixed',
    'normal_turn_1549626293857325.csv': 'normal_turn'
}

read_frames = []

for data_file, event_type in data_files.items():
    tmp_df = pd.read_csv('data/' + data_file)
    tmp_df['event_type'] = event_type
    tmp_df = tmp_df[['event_type', 'label', 'timestamp', 'accel_x', 'accel_y', 'accel_z', 'gyro_roll', 'gyro_pitch', 'gyro_yaw']]
    read_frames.append(tmp_df)

df = pd.concat(read_frames)
df.to_csv('data/unified.csv')
df.shape

(81435, 9)

# Using Spark to wrangle the data

After creating the Spark session, we will load the `data/unified.csv`.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Truck Data Wrangler").getOrCreate()
spark

## Average acceleration by event type and label

To calculate the average acceleration, grouping by event type and label, we will first read the unified.csv and create a SQL table from it:

In [3]:
truck_events_df = spark.read.csv('data/unified.csv', header=True)
truck_events_df.createOrReplaceTempView("truck_events")

After creating the SQL view, the following query will give us the grouped results we want to analyze:

In [4]:
spark.sql("""
SELECT 
    event_type, 
    label, 
    AVG(accel_x) AS avg_accel_x,
    AVG(accel_y) AS avg_accel_y,
    AVG(accel_z) AS avg_accel_z
FROM 
    truck_events 
GROUP BY
    event_type, label
ORDER BY
    event_type, label
""").toPandas()

,event_type,label,avg_accel_x,avg_accel_y,avg_accel_z
0,aggressive_longitudinal_acceleration,0,0.072664,0.059177,0.749448
1,aggressive_longitudinal_acceleration,1,0.011803,0.063492,0.749371
2,aggressive_turn,0,0.071153,0.077650,0.751435
3,aggressive_turn,1,0.051853,0.041711,0.751523
4,agressive_bump,0,0.013834,0.061374,0.749088
5,agressive_bump,1,-0.065852,0.032218,0.748155
6,normal_longitudinal_acceleration,0,0.026090,0.078436,0.751428
7,normal_mixed,0,0.026531,0.063761,0.752289
8,normal_turn,0,0.068144,0.070013,0.751433


## The maximum jerk timestamp

To calculate the maximum jerk timestamp, first of all, we have to calculate the jerk of the momentum.

Thus, to calculate the jerk we can do it with this simple equation:

`jerk = (𝑎2 − 𝑎1) / (𝑡2 − 𝑡1)`

Where:
 - 𝑎2 is the acceleration of the current sensor registry
 - 𝑎1 is the acceleration of the last sensor registry before the current
 - 𝑡2 is the timestamp of the current sensor registry
 - 𝑡1 is the timestamp of the last sensor registry before the current
 


In [5]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.window import Window

column_list = ["event_type","label"]
win_spec = Window.partitionBy([col(x) for x in column_list]).orderBy("timestamp")

jerk_truck_events_df = truck_events_df

columns_that_needs_latest_values = ['accel_x', 'accel_y', 'accel_z', 'timestamp']

for column_name in columns_that_needs_latest_values:
    jerk_truck_events_df = jerk_truck_events_df.withColumn("last_" + column_name, F.lag(col(column_name)).over(win_spec))

jerk_truck_events_df.toPandas().head()

,_c0,event_type,label,timestamp,accel_x,accel_y,accel_z,gyro_roll,gyro_pitch,gyro_yaw,last_accel_x,last_accel_y,last_accel_z,last_timestamp
0,0,aggressive_longitudinal_acceleration,0,1549653321089461,0.0517081543803215,0.0543818324804306,0.7490020394325256,0.00022493861615657806,0.00021865405142307281,-0.0003485679626464844,None,None,None,None
1,1,aggressive_longitudinal_acceleration,0,1549653321139461,0.05285722389817238,0.0543818324804306,0.7488265037536621,0.00022482685744762418,0.0002185441553592682,-0.00034839287400245667,0.0517081543803215,0.0543818324804306,0.7490020394325256,1549653321089461
2,2,aggressive_longitudinal_acceleration,0,1549653321189461,0.053431760519742966,0.05732139199972153,0.7502307891845703,0.00022471509873867035,-0.0003134123980998993,0.0001836288720369339,0.05285722389817238,0.0543818324804306,0.7488265037536621,1549653321139461
3,3,aggressive_longitudinal_acceleration,0,1549653321239461,0.05228269100189209,0.05585161224007607,0.7486509680747986,-0.00030724331736564636,0.00021859072148799896,0.0007153842598199843,0.053431760519742966,0.05732139199972153,0.7502307891845703,1549653321189461
4,4,aggressive_longitudinal_acceleration,0,1549653321289461,0.05228269100189209,0.05732139199972153,0.7488265037536621,-0.00030709058046340937,0.00021848082542419442,-0.0003486666828393936,0.05228269100189209,0.05585161224007607,0.7486509680747986,1549653321239461


Now that we have the latest values appended on the data frame, we need to create a last column with the jerk value calculated:

In [17]:
# x axis
df = jerk_truck_events_df.withColumn(
    "jerk_x", 
    F.when(F.isnull(col("last_accel_x")), 0)
     .when(F.isnull(col("last_timestamp")), 0)
     .otherwise((col("accel_x") - col("last_accel_x")) / (col("timestamp") - col("last_timestamp")))
)

# y axis
df = df.withColumn(
    "jerk_y", 
    F.when(F.isnull(col("last_accel_y")), 0)
     .when(F.isnull(col("last_timestamp")), 0)
     .otherwise((col("accel_y") - col("last_accel_y")) / (col("timestamp") - col("last_timestamp")))
)

# z axis
df = df.withColumn(
    "jerk_z", 
    F.when(F.isnull(col("last_accel_z")), 0)
     .when(F.isnull(col("last_timestamp")), 0)
     .otherwise((col("accel_z") - col("last_accel_z")) / (col("timestamp") - col("last_timestamp")))
)

#df.toPandas().head()
df.describe("jerk_x").show()
df.describe("jerk_y").show()
df.describe("jerk_z").show()

+-------+--------------------+
|summary|              jerk_x|
+-------+--------------------+
|  count|               81435|
|   mean|1.441581488571492...|
| stddev|2.072853275067905...|
|    min|-4.46823395474348...|
|    max|3.535190045833587E-5|
+-------+--------------------+

+-------+--------------------+
|summary|              jerk_y|
+-------+--------------------+
|  count|               81435|
|   mean|3.967448509129019...|
| stddev|3.500832049863185...|
|    min|-7.15935850143432...|
|    max|6.377410769462586E-5|
+-------+--------------------+

+-------+--------------------+
|summary|              jerk_z|
+-------+--------------------+
|  count|               81435|
|   mean|7.190501408417495...|
| stddev|4.118166978012494...|
|    min|-9.21902656555175...|
|    max|1.050393342971801...|
+-------+--------------------+



#### Answering the question, the of the maximum jerk timestamp (per event_type/label):

In [28]:
df.createOrReplaceTempView("jerked_truck_events")

spark.sql("""
SELECT 
    event_type, 
    label, 
    from_unixtime(cast((timestamp / 1000) as bigint),'yyyy-MM-dd HH:mm:ss.SSS') AS timestamp,
    jerk_x
FROM 
    jerked_truck_events 
ORDER BY
    jerk_x DESC
LIMIT 1
""").toPandas()

,event_type,label,timestamp,jerk_x
0,agressive_bump,1,51092-09-20 17:47:48.000,0.000035


In [21]:

spark.sql("""
SELECT 
    event_type, 
    label, 
    from_unixtime(cast((timestamp / 1000) as bigint),'yyyy-MM-dd HH:mm:ss.SSS') AS timestamp,
    jerk_y
FROM 
    jerked_truck_events 
ORDER BY
    jerk_y DESC, event_type, label
LIMIT 1
""").toPandas()

,event_type,label,timestamp,jerk_y
0,agressive_bump,1,51092-09-19 07:45:18.000,0.000064


In [22]:

spark.sql("""
SELECT 
    event_type, 
    label, 
    from_unixtime(cast((timestamp / 1000) as bigint),'yyyy-MM-dd HH:mm:ss.SSS') AS timestamp,
    jerk_z
FROM 
    jerked_truck_events 
ORDER BY
    jerk_z DESC, event_type, label
LIMIT 1
""").toPandas()

,event_type,label,timestamp,jerk_z
0,agressive_bump,1,51092-09-18 03:45:18.000,0.000011
